# "Moving Pictures" tutorial

**_note:_ This guide assumes you have QIIME 2 installed (e.g. using this [procedure](https://docs.qiime2.org/2019.10/install/native/)). To execute the script properly, open this notebook in a Jupyter Notebook from within a conda QIIME 2 environment.**

**_note:_ This tutorial is an adaptation of the same tutorial that may be found on the [official QIIME 2 docs website](https://docs.qiime2.org/2019.10/tutorials/moving-pictures/). The original tutorial uses the QIIME 2 CLI interface.**

Instead of CLI interface, this tutorial uses [Artifact API](https://docs.qiime2.org/2019.10/interfaces/artifact-api/) - a Python 3 application programmer interface (API) for QIIME 2. The Artifact API supports interactive computing with QIIME 2 using the Python 3 programming language. The API is automatically generated, and its availability depends on which QIIME 2 plugins are currently installed. It has been optimized for use in the Jupyter Notebook. The Artifact API is a part of the QIIME 2 framework; no additional software needs to be installed to use it. 

**The notebook was tested using the `2019.10` version of QIIME 2.**

## Before you start: close this notebook and Jupyter session, and run `jupyter serverextension enable --py qiime2 --sys-prefix`. Then, restart this notebook.

In this tutorial you’ll use QIIME 2 to perform an analysis of human microbiome samples from two individuals at four body sites at five timepoints, the first of which immediately followed antibiotic usage. A study based on these samples was originally published in [Caporaso et al. (2011)](https://www.ncbi.nlm.nih.gov/pubmed/21624126). The data used in this tutorial were sequenced on an Illumina HiSeq using the [Earth Microbiome Project](http://earthmicrobiome.org/) hypervariable region 4 (V4) 16S rRNA sequencing protocol.

> Question 1: Why is 16S rRNA marker gene sequencing so often used to study the microbiome? What other marker genes are commonly used to identify Bacteria and Archaea? What marker genes do we use to identify other microbial kingdoms (e.g. Fungi and microbial Eukaryotes)?

Let's import all necessary modules

In [ ]:
%matplotlib inline
import qiime2
from tempfile import mkdtemp
from qiime2.plugins import demux, dada2, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition

Before beginning this tutorial, create a new directory and change to that directory. The following two commands should do it for you. 

#### make sure to change the path in workdir

In [ ]:
workdir='/path/to/your/directory/'

In [ ]:
!mkdir -p $workdir
!cd $workdir

## Sample metadata

Before starting the analysis, explore the sample metadata to familiarize yourself with the samples used in this study. The [sample metadata](https://docs.google.com/spreadsheets/d/16oomVnULW-uesehNZc_mKIDTnuRoTiun0CpzSFZagvo/edit#gid=0) is available as a Google Sheet. You can download this file as tab-separated text by selecting `File` > `Download as` > `Tab-separated values`. Alternatively, the following command will download the sample metadata as tab-separated text and save it in the file `sample-metadata.tsv`. This `sample-metadata.tsv` file is used throughout the rest of the tutorial.

In [ ]:
!wget -O $workdir/"sample-metadata.tsv" "https://data.qiime2.org/2019.10/tutorials/moving-pictures/sample_metadata.tsv"

## Obtaining and importing data

Download the sequence reads that we’ll use in this analysis. In this tutorial we’ll work with a small subset of the complete sequence data so that the commands will run quickly.

_Make new directory for the archives to be downloaded._

In [ ]:
!mkdir $workdir/emp-single-end-sequences

Download `barcodes.fastq.gz` archive containing barcode sequances identifying individual samples.

In [ ]:
!wget -O $workdir/"emp-single-end-sequences/barcodes.fastq.gz" "https://data.qiime2.org/2019.10/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"

Download `sequences.fastq.gz` archive containing all the reads sequanced.

In [ ]:
!wget -O $workdir/"emp-single-end-sequences/sequences.fastq.gz" "https://data.qiime2.org/2019.10/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"

### Import data as a qiime2 artifact

All data that is used as input to QIIME 2 is in form of QIIME 2 artifacts, which contain information about the type of data and the source of the data. So, the first thing we need to do is import these sequence data files into a QIIME 2 artifact.

The semantic type of this QIIME 2 artifact is `EMPSingleEndSequences`. `EMPSingleEndSequences` QIIME 2 artifacts contain sequences that are multiplexed, meaning that the sequences have not yet been assigned to samples (hence the inclusion of both `sequences.fastq.gz` and `barcodes.fastq.gz` files, where the `barcodes.fastq.gz` contains the barcode read associated with each sequence in `sequences.fastq.gz.`) 


In [ ]:
single_end_sequences = qiime2.Artifact.import_data('EMPSingleEndSequences', workdir+'/emp-single-end-sequences/')

### Read metadata into `qiime2.Metadata` object

In [ ]:
sample_metadata = qiime2.Metadata.load(workdir+'/sample-metadata.tsv')

## Demultiplexing sequences 

> Question 2: What is demultiplexing and why do we need to demultiplex? 

To demultiplex sequences we need to know which barcode sequence is associated with each sample. This information is contained in the [sample metadata](https://docs.google.com/spreadsheets/d/16oomVnULW-uesehNZc_mKIDTnuRoTiun0CpzSFZagvo/edit#gid=0) file. You can run the following commands to demultiplex the sequences (the `demux.methods.emp-single` function refers to the fact that these sequences are barcoded according to the [Earth Microbiome Project](http://earthmicrobiome.or-g/) protocol, and are single-end reads). The `demux.qza` QIIME 2 artifact will contain the demultiplexed sequences.

In [ ]:
demux_sequences = demux.methods.emp_single(single_end_sequences,
                                           sample_metadata.get_column('barcode-sequence'))

After demultiplexing, it’s useful to generate a summary of the demultiplexing results. This allows you to determine how many sequences were obtained per sample, and also to get a summary of the distribution of sequence qualities at each position in your sequence data.

In [ ]:
demux_summary = demux.visualizers.summarize(demux_sequences.per_sample_sequences)
demux_summary.visualization


## Sequence quality control and feature table construction

QIIME 2 plugins are available for several quality control methods, including DADA2, Deblur, and basic quality-score-based filtering. In this tutorial we present this step using [DADA2](https://www.ncbi.nlm.nih.gov/pubmed/27214047) and [Deblur](https://msystems.asm.org/content/2/2/e00191-16). These steps are interchangeable, so you can use whichever of these you prefer. The result of both of these methods will be a `FeatureTable[Frequency]` QIIME 2 artifact, which contains counts (frequencies) of each unique sequence in each sample in the dataset, and a `FeatureData[Sequence]` QIIME 2 artifact, which maps feature identifiers in the `FeatureTable` to the sequences they represent.

## Option 1: DADA2

[DADA2](https://www.ncbi.nlm.nih.gov/pubmed/27214047) is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data. As implemented in the `q2-dada2` plugin, this quality control process will additionally filter any phiX reads (commonly present in marker gene Illumina sequence data) that are identified in the sequencing data, and will filter chimeric sequences.

The `dada2.methods.denoise_single` method requires two parameters that are used in quality filtering: `trim_left`, which trims off the first `m` bases of each sequence, and `trunc_len` which truncates each sequence at position `n`. This allows the user to remove low quality regions of the sequences. To determine what values to pass for these two parameters, you should review the Interactive Quality Plot tab in the `demux.qzv` file that was generated by qiime demux summarize above.

In the `demux.qzv` quality plots, we see that the quality of the initial bases seems to be high, so we won’t trim any bases from the beginning of the sequences. The quality seems to drop off around position 120, so we’ll truncate our sequences at 120 bases. This next command may take up to 10 minutes to run, and is the slowest step in this tutorial.


In [ ]:
denoised_sequences = dada2.methods.denoise_single(demux_sequences.per_sample_sequences, trim_left = 0, \
trunc_len = 120)

In [ ]:
denoise_stats = metadata.visualizers.tabulate(denoised_sequences.denoising_stats.view(qiime2.Metadata))
denoise_stats.visualization

**If you want to stay with Option 1, go straight to FeatureTable and FeatureData summaries. If you want to try Option 2, denoising sequances with Deblur, proceed to the next cell.**

## Option 2: Deblur

Deblur uses sequence error profiles to associate erroneous sequence reads with the true biological sequence from which they are derived, resulting in high quality sequence variant data. This is applied in two steps. First, an initial quality filtering process based on quality scores is applied. This method is an implementation of the quality filtering approach described by [Bokulich et al. (2013)](https://www.nature.com/articles/nmeth.2276).

In [ ]:
demux_filter_stats = quality_filter.methods.q_score(demux_sequences.per_sample_sequences)

Next, the Deblur workflow is applied using the `deblur.methods.denoise-16S method`. This method requires one parameter that is used in quality filtering, `trim_length = n` which truncates the sequences at position `n`. In general, the Deblur developers recommend setting this value to a length where the median quality score begins to drop too low. On these data, the quality plots (prior to quality filtering) suggest a reasonable choice is in the 115 to 130 sequence position range. This is a subjective assessment. One situation where you might deviate from that ecommendation is when performing a meta-analysis across multiple sequencing runs. In this type of meta-analysis, it is critical that the read lengths be the same for all of the sequencing runs being compared to avoid introducing a study-specific bias. Since 120 is reasonable given the quality plots, we’ll pass `trim_length` parameter the value of 120. This next command may take up to 10 minutes to run.

In [ ]:
denoised_sequences = deblur.methods.denoise_16S(demux_sequences.per_sample_sequences,
                                             trim_length=120,
                                              sample_stats=True)

The two commands used in this section generate QIIME 2 artifacts containing summary statistics. To view those summary statistics, you can visualize them using `metadata.visualizers.tabulate` and `deblur.visulaizers.visualize-stats`, respectively.
> Question 3: Look at generated filtering stats after demultiplexing. Pay attention to columns names. What is the meaning of `reads-too-short-after-truncation` and `reads-exceeding-maximum-ambiguous-bases` columns in the context of filtering?

> Question 4: Look at statistics visulization after appling denoising using Deblur. How do you understand `reads-missed-reference`, `reads-hit-artifact`, `reads-derep`, `reads-chimeric` in context of methodology (sequancing errors, using reference data bases)?

In [ ]:
filter_stats = metadata.visualizers.tabulate(demux_filter_stats.filter_stats.view(qiime2.Metadata))
filter_stats.visualization

In [ ]:
denoise_stats = deblur.visualizers.visualize_stats(denoised_sequences.stats)
denoise_stats.visualization


## FeatureTable and FeatureData summaries

After the quality filtering step completes, you’ll want to explore the resulting data. You can do this using the following two commands, which will create visual summaries of the data. The `feature_table.visulizers.summarize` function will give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics. The `feature_table.visulizers.tabulate_seqs` command will provide a mapping of feature IDs to sequences, and provide links to easily BLAST each sequence against the NCBI nt database. The latter visualization will be very useful later in the tutorial, when you want to learn more about specific features that are important in the data set.


In [ ]:
output_viz = feature_table.visualizers.summarize(denoised_sequences.table)
output_viz.visualization

In [ ]:
denoise_feature_table_summary = feature_table.visualizers.tabulate_seqs(denoised_sequences.representative_sequences)
denoise_feature_table_summary.visualization


## Generate a tree for phylogenetic diversity analyses

QIIME supports several phylogenetic diversity metrics, including Faith’s Phylogenetic Diversity and weighted and unweighted UniFrac. In addition to counts of features per sample (i.e., the data in the `FeatureTable[Frequency]` QIIME 2 artifact), these metrics require a rooted phylogenetic tree relating the features to one another. This information will be stored in a `Phylogeny[Rooted]` QIIME 2 artifact. To generate a phylogenetic tree we will use `phylogeny.pipelines.align_to_tree_mafft_fasttree` pipeline.

First, the pipeline uses the `mafft` program to perform a multiple sequence alignment of the sequences in our `FeatureData[Sequence]` to create a `FeatureData[AlignedSequence]` QIIME 2 artifact. Next, the pipeline masks (or filters) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree. Following that, the pipeline applies FastTree to generate a phylogenetic tree from the masked alignment. The FastTree program creates an unrooted tree, so in the final step in this section midpoint rooting is applied to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.

In [ ]:
rooted_tree = phylogeny.pipelines.align_to_tree_mafft_fasttree(denoised_sequences.representative_sequences)

## Alpha and beta diversity analysis

QIIME 2’s diversity analyses are available through the `q2-diversity` plugin, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations. We’ll first apply the `diversity.pipelines.core_metrics_phylogenetic` method, which rarefies a `FeatureTable[Frequency]` to a user-specified depth, computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics. The metrics computed by default are:

* Alpha diversity
    * Shannon’s diversity index (a quantitative measure of community richness)
    * Observed OTUs (a qualitative measure of community richness)
    * Faith’s Phylogenetic Diversity (a qualitiative measure of community richness that incorporates phylogenetic relationships between the features)
    * Evenness (or Pielou’s Evenness; a measure of community evenness)

* Beta diversity
    * Jaccard distance (a qualitative measure of community dissimilarity)
    * Bray-Curtis distance (a quantitative measure of community dissimilarity)
    * unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)
    * weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

An important parameter that needs to be provided to this script is `sampling_depth`, which is the even sampling (i.e. rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter. For example, if you provide `sampling_depth = 500`, this step will subsample the counts in each sample without replacement so that each sample in the resulting table has a total count of 500. If the total count for any sample(s) are smaller than this value, those samples will be dropped from the diversity analysis. Choosing this value is tricky. We recommend making your choice reviewing the information presented in the table that was created above. Choose a value that is as high as possible (so you retain more sequences per sample) while excluding as few samples as possible. 
> Question 5: View the above mentioned QIIME 2 artifact containing table, and in particular the _Interactive Sample Detail_ tab in that visualization. What value would you choose to pass for  `sampling_depth`? How many samples will be excluded from your analysis based on this choice? How many total sequences will you be analyzing in the  `core_metrics_phylogenetic ` pipeline?

In [ ]:
core_metrics = diversity.pipelines.core_metrics_phylogenetic(table = denoised_sequences.table,
                                                             phylogeny = rooted_tree.rooted_tree,
                                                             sampling_depth = 1103,
                                                             metadata = sample_metadata)

Here we set the `sampling_depth` parameter to 1103. This value was chosen based on the number of sequences in the `L3S313` sample because it’s close to the number of sequences in the next few samples that have higher sequence counts, and because it is considerably higher (relatively) than the number of sequences in the samples that have fewer sequences. This will allow us to retain most of our samples. The three samples that have fewer sequences will be dropped from the `core_metrics_phylogenetic` analyses and anything that uses these results. It is worth noting that all three of these samples are “right palm” samples. Losing a disproportionate number of samples from one metadata category is not ideal. However, we are dropping a small enough number of samples here that this felt like the best compromise between total sequences analyzed and number of samples retained.


### alpha group significance

After computing diversity metrics, we can begin to explore the microbial composition of the samples in the context of the sample metadata. This information is present in the sample metadata file that was downloaded earlier.

We’ll first test for associations between categorical metadata columns and alpha diversity data. We’ll do that here for the Faith Phylogenetic Diversity (a measure of community richness) and evenness metrics.

In [ ]:
faith_pd_group_significance = diversity.actions.alpha_group_significance(core_metrics.faith_pd_vector,
                                                                         sample_metadata)

In [ ]:
evenness_group_significance = diversity.actions.alpha_group_significance(core_metrics.evenness_vector,
                                                                         sample_metadata)


Visualize group significance results

In [ ]:
faith_pd_group_significance.visualization

In [ ]:
evenness_group_significance.visualization


> Question 6: Which categorical sample metadata columns are most strongly associated with the differences in microbial community richness? Are these differences statistically significant? 

> Question 7: Which categorical sample metadata columns are most strongly associated with the differences in microbial community evenness? Are these differences statistically significant? 
### beta group significance

In this data set, no continuous sample metadata columns (e.g., `days-since-experiment-start`) are correlated with alpha diversity, so we won’t test for those associations here. If you’re interested in performing those tests (for this data set, or for others), you can use the `diversity.visualizers.alpha-correlation` command.

Next we’ll analyze sample composition in the context of categorical metadata using PERMANOVA (first described in [Anderson (2001)](https://onlinelibrary.wiley.com/doi/full/10.1111/j.1442-9993.2001.01070.pp.x)) using the `diversity.actions.beta_group_significance` method. The following commands will test whether distances between samples within a group, such as samples from the same body site (e.g., gut), are more similar to each other then they are to samples from the other groups (e.g., tongue, left palm, and right palm). It will also perform pairwise tests that will allow you to determine which specific pairs of groups (e.g., tongue and gut) differ from one another, if any. This command can be slow to run, especially when running pairwise analysis, since it is based on permutation tests. So, unlike the previous commands, we’ll run this on specific columns of metadata that we’re interested in exploring, rather than all metadata columns that it’s applicable to. Here we’ll apply this to our unweighted UniFrac distances, using two sample metadata columns, as follows.

In [ ]:
uUniFrac_BodySite_significance = diversity.actions.beta_group_significance(core_metrics.unweighted_unifrac_distance_matrix,
                                                                           sample_metadata.get_column('body-site'))

In [ ]:
uUniFrac_Subject_significance = diversity.actions.beta_group_significance(core_metrics.unweighted_unifrac_distance_matrix,
                                                                          sample_metadata.get_column('subject'))

In [ ]:
uUniFrac_BodySite_significance.visualization

In [ ]:
uUniFrac_Subject_significance.visualization

> Question 8: Are the associations between subjects and differences in microbial composition statistically significant? How about body sites? What specific pairs of body sites are significantly different from each other?  

Again, none of the continuous sample metadata that we have for this data set are correlated with sample composition, so we won’t test for those associations here. If you’re interested in performing those tests, you can use the `metadata.visualizers.distance_matrix` in combination with `diversity.visualizers.mantel` and `diversity.visualizers.bioenv` commands.

Finally, ordination is a popular approach for exploring microbial community composition in the context of sample metadata. We can use the [Emperor](http://biocore.github.io/emperor/) tool to explore principal coordinates (PCoA) plots in the context of sample metadata. While our `diversity.pipelines.core_metrics_phylogenetic` command did already generate some Emperor plots, we want to pass an optional parameter, `custom_axes`, which is very useful for exploring time series data. The PCoA results that were used in `core-metrics-phylogeny` are also available, making it easy to generate new visualizations with Emperor. We will generate Emperor plots for unweighted UniFrac and Bray-Curtis so that the resulting plot will contain axes for principal coordinate 1, principal coordinate 2, and days since the experiment start. We will use that last axis to explore how these samples changed over time.

In [ ]:
emperor_plot_unifrac = emperor.visualizers.plot(core_metrics.unweighted_unifrac_pcoa_results,
                                                sample_metadata,
                                                custom_axes=['days-since-experiment-start'])
emperor_plot_unifrac.visualization

In [ ]:
emperor_plot_bc = emperor.visualizers.plot(core_metrics.bray_curtis_pcoa_results,
                                           sample_metadata,
                                           custom_axes=['days-since-experiment-start'])
emperor_plot_bc.visualization

> Question 9: What differences do you observe between the unweighted UniFrac and Bray-Curtis PCoA plots? 

## Alpha rarefaction plotting

In this section we’ll explore alpha diversity as a function of sampling depth using the `diversity.actions.alpha_rarefaction` visualizer. This visualizer computes one or more alpha diversity metrics at multiple sampling depths, in steps between 1 (optionally controlled with `min_depth` and the value provided as `max_depth`. At each sampling depth step, 10 rarefied tables will be generated, and the diversity metrics will be computed for all samples in the tables. The number of iterations (rarefied tables computed at each sampling depth) can be controlled with `iterations`. Average diversity values will be plotted for each sample at each even sampling depth, and samples can be grouped based on metadata in the resulting visualization if sample metadata is provided with the `metadata` parameter.

In [ ]:
alpha_rarefaction = diversity.actions.alpha_rarefaction(table = denoised_sequences.table,
                                                        max_depth = 4000,
                                                        phylogeny = rooted_tree.rooted_tree,
                                                        metadata = sample_metadata)

In [ ]:
alpha_rarefaction.visualization

The visualization will have two plots. The top plot is an alpha rarefaction plot, and is primarily used to determine if the richness of the samples has been fully observed or sequenced. If the lines in the plot appear to “level out” (i.e., approach a slope of zero) at some sampling depth along the x-axis, that suggests that collecting additional sequences beyond that sampling depth would not be likely to result in the observation of additional features. If the lines in a plot don’t level out, this may be because the richness of the samples hasn’t been fully observed yet (because too few sequences were collected), or it could be an indicator that a lot of sequencing error remains in the data (which is being mistaken for novel diversity).

The bottom plot in this visualization is important when grouping samples by metadata. It illustrates the number of samples that remain in each group when the feature table is rarefied to each sampling depth. If a given sampling depth `d` is larger than the total frequency of a sample `s` (i.e., the number of sequences that were obtained for sample `s`), it is not possible to compute the diversity metric for sample s at sampling depth `d`. If many of the samples in a group have lower total frequencies than `d`, the average diversity presented for that group at `d` in the top plot will be unreliable because it will have been computed on relatively few samples. When grouping samples by metadata, it is therefore essential to look at the bottom plot to ensure that the data presented in the top plot is reliable.

> Question 10: When grouping samples by “body-site” and viewing the alpha rarefaction plot for the “observed_otus” metric, which body sites (if any) appear to exhibit sufficient diversity coverage (i.e., their rarefaction curves level off)? How many sequence variants appear to be present in those body sites? 

> Question 11: When grouping samples by “body-site” and viewing the alpha rarefaction plot for the “observed_otus” metric, the line for the “right palm” samples appears to level out at about 40, but then jumps to about 140. What do you think is happening here? (Hint: be sure to look at both the top and bottom plots.) 

## Taxonomic analysis

In the next sections we’ll begin to explore the taxonomic composition of the samples, and again relate that to sample metadata. The first step in this process is to assign taxonomy to the sequences in our `FeatureData[Sequence]` QIIME 2 artifact. We’ll do that using a pre-trained Naive Bayes classifier and the `q2-feature-classifier` plugin. This classifier was trained on the Greengenes 13_8 99% OTUs, where the sequences have been trimmed to only include 250 bases from the region of the 16S that was sequenced in this analysis (the V4 region, bound by the 515F/806R primer pair). We’ll apply this classifier to our sequences, and we can generate a visualization of the resulting mapping from sequence to taxonomy.

In [ ]:
!wget -O $workdir"/gg-13-8-99-515-806-nb-classifier.qza" "https://data.qiime2.org/2019.10/common/gg-13-8-99-515-806-nb-classifier.qza"

In [ ]:
gg_classifier = qiime2.Artifact.load(workdir+'/gg-13-8-99-515-806-nb-classifier.qza')

In [ ]:
taxonomy = feature_classifier.methods.classify_sklearn(reads = denoised_sequences.representative_sequences,
                                                       classifier = gg_classifier)

In [ ]:
taxonomy_classification = metadata.visualizers.tabulate(taxonomy.classification.view(qiime2.Metadata))
taxonomy_classification.visualization

Next, we can view the taxonomic composition of our samples with interactive bar plots. Generate those plots with the following command and then open the visualization.

In [ ]:
taxa_bar_plot = taxa.visualizers.barplot(denoised_sequences.table, taxonomy.classification, sample_metadata)
taxa_bar_plot.visualization

> Question 12: Visualize the samples at Level 2 (which corresponds to the phylum level in this analysis), and then sort the samples by `body-site`, then by `subject`, and then by `days-since-experiment-start`. What are the dominant phyla in each in `body-site`? Do you observe any consistent change across the two subjects between `days-since-experiment-start` `0` and the later timepoints? 

## Differential abundance testing with ANCOM

ANCOM can be applied to identify features that are differentially abundant (i.e. present in different abundances) across sample groups. As with any bioinformatics method, you should be aware of the assumptions and limitations of ANCOM before using it. We recommend reviewing the [ANCOM paper](https://www.ncbi.nlm.nih.gov/pubmed/26028277) before using this method.

**Note  
Differential abundance testing in microbiome analysis is an active area of research. There are two QIIME 2 plugins that can be used for this: `q2-gneiss` and `q2-composition`. This section uses `q2-composition`, but there is [another tutorial which uses gneiss](https://docs.qiime2.org/2019.10/tutorials/gneiss/) on a different dataset if you are interested in learning more.**

ANCOM is implemented in the `q2-composition` plugin. ANCOM assumes that few (less than about 25%) of the features are changing between groups. If you expect that more features are changing between your groups, you should not use ANCOM as it will be more error-prone (an increase in both Type I and Type II errors is possible). Because we expect a lot of features to change in abundance across body sites, in this tutorial we’ll filter our full feature table to only contain gut samples. We’ll then apply ANCOM to determine which, if any, sequence variants and genera are differentially abundant across the gut samples of our two subjects.

We’ll start by creating a feature table that contains only the gut samples. (To learn more about filtering, see the [Filtering Data](https://docs.qiime2.org/2019.10/tutorials/filtering/) tutorial.)

In [ ]:
gut_denoise = feature_table.methods.filter_samples(denoised_sequences.table,
                                                  metadata = sample_metadata,
                                                  where = "[body-site]='gut'")

ANCOM operates on a `FeatureTable[Composition]` QIIME 2 artifact, which is based on frequencies of features on a per-sample basis, but cannot tolerate frequencies of zero. To build the composition artifact, a `FeatureTable[Frequency]` artifact must be provided to add-pseudocount (an imputation method), which will produce the `FeatureTable[Composition]` artifact.

In [ ]:
gut_denoise_composition = composition.actions.add_pseudocount(gut_denoise.filtered_table)

We can then run ANCOM on the `subject` column to determine what features differ in abundance across the gut samples of the two subjects.

In [ ]:
ancom_gut_denoise = composition.actions.ancom(table = gut_denoise_composition.composition_table,
                                             metadata = sample_metadata.get_column('subject'))

In [ ]:
ancom_gut_denoise.visualization

>Question 13: Which sequence variants differ in abundance across Subject? In which subject is each sequence variant more abundant? What are the taxonomies of some of these sequence variants? (To answer the last question you’ll need to refer to another visualization that was generated in this tutorial.) 
We’re also often interested in performing a differential abundance test at a specific taxonomic level. To do this, we can collapse the features in our `FeatureTable[Frequency]` at the taxonomic level of interest, and then re-run the above steps. In this tutorial, we collapse our feature table at the genus level (i.e. level 6 of the Greengenes taxonomy).

In [ ]:
gut_table_l6 = taxa.methods.collapse(table = gut_denoise.filtered_table,
                                     taxonomy = taxonomy.classification,
                                     level = 6)

In [ ]:
gut_table_l6_composition = composition.actions.add_pseudocount(gut_table_l6.collapsed_table)

In [ ]:
ancom_gut_table_l6 = composition.actions.ancom(table = gut_table_l6_composition.composition_table,
                                               metadata = sample_metadata.get_column('subject'))

In [ ]:
ancom_gut_table_l6.visualization

>Question 14: Which genera differ in abundance across subject? In which subject is each genus more abundant?